# Preferential Bayesian Optimization: RANDOM
This notebook demonstrates the use of random querying on ordinal (preference) data.

Formulation by Nguyen Quoc Phong.

In [1]:
import numpy as np
import gpflow
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import os
import pickle

from gpflow.utilities import set_trainable, print_summary
gpflow.config.set_default_summary_fmt("notebook")

sys.path.append(os.path.split(os.path.split(os.path.split(os.getcwd())[0])[0])[0]) # Move 3 levels up directory to import PBO
import PBO

In [2]:
objective = PBO.objectives.hartmann3d
objective_low = 0
objective_high = 1.
objective_name = "Hart3"
acquisition_name = "RANDOM"
experiment_name = "PBO" + "_" + acquisition_name + "_" + objective_name

In [3]:
num_runs = 20
num_evals = 20
num_samples = 1000
num_choices = 2
input_dims = 3
num_maximizers = 20
num_init_points = 3
num_inducing_init = 3
num_discrete_per_dim = 100 # Discretization of continuous input space

In [4]:
results_dir = os.getcwd() + '/results/' + experiment_name + '/'

try:
    # Create target Directory
    os.makedirs(results_dir)
    print("Directory " , results_dir ,  " created ") 
except FileExistsError:
    print("Directory " , results_dir ,  " already exists")

Directory  /home/sebtsh/PBO/notebooks/RANDOM/results/PBO_RANDOM_Hart3/  already exists


In [5]:
def get_noisy_observation(X, objective):
    f = PBO.objectives.objective_get_f_neg(X, objective)
    return PBO.observation_model.gen_observation_from_f(X, f, 1)

In [6]:
def train_and_visualize(X, y, num_inducing, title):
    
    # Train model with data
    q_mu, q_sqrt, u, inputs, k, indifference_threshold = PBO.models.learning_stochastic.train_model_fullcov(X, y, 
                                                                         num_inducing=num_inducing,
                                                                         obj_low=objective_low,
                                                                         obj_high=objective_high,
                                                                         num_steps=3000)
    likelihood = gpflow.likelihoods.Gaussian()
    model = PBO.models.learning.init_SVGP_fullcov(q_mu, q_sqrt, u, k, likelihood)
    u_mean = q_mu.numpy()
    inducing_vars = u.numpy()
    
    return model, inputs, u_mean, inducing_vars

In [7]:
def uniform_grid(input_dims, num_discrete_per_dim, low=0., high=1.):
    """
    Returns an array with all possible permutations of discrete values in input_dims number of dimensions.
    :param input_dims: int
    :param num_discrete_per_dim: int
    :param low: int
    :param high: int
    :return: tensor of shape (num_discrete_per_dim ** input_dims, input_dims)
    """
    num_points = num_discrete_per_dim ** input_dims
    out = np.zeros([num_points, input_dims])
    discrete_points = np.linspace(low, high, num_discrete_per_dim)
    for i in range(num_points):
        for dim in range(input_dims):
            val = num_discrete_per_dim ** (dim)
            out[i, dim] = discrete_points[int((i // val) % num_discrete_per_dim)]
    return out

This function is our main metric for the performance of the acquisition function: The closer the model's best guess to the global minimum, the better.

In [8]:
def best_guess(model):
    """
    Returns a GP model's best guess of the global maximum of f.
    """
    xx = uniform_grid(input_dims, num_discrete_per_dim, low=objective_low, high=objective_high)
    res = model.predict_f(xx)[0].numpy()
    return xx[np.argmax(res)]

Store the results in these arrays:

In [9]:
num_data_at_end = int((num_init_points-1) * num_init_points / 2 + num_evals)
X_results = np.zeros([num_runs, num_data_at_end, num_choices, input_dims])
y_results = np.zeros([num_runs, num_data_at_end, 1, input_dims])
best_guess_results = np.zeros([num_runs, num_evals, input_dims])

Create the initial values for each run:

In [10]:
np.random.seed(0)
init_points = np.random.uniform(low=objective_low, high=objective_high, size=[num_runs, num_init_points, input_dims])
num_combs = int((num_init_points-1) * num_init_points / 2)
init_vals = np.zeros([num_runs, num_combs, num_choices, input_dims])
for run in range(num_runs):
    cur_idx = 0
    for init_point in range(num_init_points-1):
        for next_point in range(init_point+1, num_init_points):
            init_vals[run, cur_idx, 0] = init_points[run, init_point]
            init_vals[run, cur_idx, 1] = init_points[run, next_point]
            cur_idx += 1

The following loops carry out the Bayesian optimization algorithm over a number of runs, with a fixed number of evaluations per run.

In [11]:
for run in range(num_runs):
    print("Beginning run %s" % (run))
    
    X = init_vals[run]
    y = get_noisy_observation(X, objective)
    
    model, inputs, u_mean, inducing_vars = train_and_visualize(X, y, num_inducing_init, "Run_{}:_Initial_model".format(run))

    for evaluation in range(num_evals):
        print("Beginning evaluation %s" % (evaluation)) 
        
        existing_idx = np.random.randint(0, inputs.shape[0])
        existing_input = inputs[existing_idx]
        random_input = np.random.uniform(low=objective_low, 
                                         high=objective_high, 
                                         size=(1, input_dims))
        
        next_query = np.zeros((num_choices, input_dims))
        next_query[0, :] = existing_input
        next_query[1, :] = random_input
        print("Evaluation %s: Next query is %s" % (evaluation, next_query))

        X = np.concatenate([X, [next_query]])
        # Evaluate objective function
        y = np.concatenate([y, get_noisy_observation(np.expand_dims(next_query, axis=0), objective)], axis=0)
        
        print("Evaluation %s: Training model" % (evaluation))
        model, inputs, u_mean, inducing_vars = train_and_visualize(X, y, 
                                                                   num_inducing_init + evaluation + 1, 
                                                                   "Run_{}_Evaluation_{}".format(run, evaluation))

        best_guess_results[run, evaluation, :] = best_guess(model)

    X_results[run] = X
    y_results[run] = y

Beginning run 0
Indifference_threshold is trainable.
Instructions for updating:
Use tf.identity instead.
Negative ELBO at step 0: 6.746526885444059 in 0.1875s
Negative ELBO at step 500: 2.196172119500443 in 52.9890s
Negative ELBO at step 1000: 2.1388190091271486 in 52.0348s
Negative ELBO at step 1500: 2.163648947696851 in 53.2397s
Negative ELBO at step 2000: 2.082518778753918 in 64.2289s
Negative ELBO at step 2500: 2.1330662592808673 in 58.1043s
Beginning evaluation 0
Evaluation 0: Next query is [[0.43758721 0.891773   0.96366276]
 [0.21874937 0.56957353 0.45210904]]
Evaluation 0: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 59.14407296532674 in 0.2114s
Negative ELBO at step 500: 4.225254189337748 in 81.3669s
Negative ELBO at step 1000: 3.5865268706736684 in 72.6612s
Negative ELBO at step 1500: 3.0044144775964785 in 78.5960s
Negative ELBO at step 2000: 2.8350340428187324 in 76.1045s
Negative ELBO at step 2500: 2.7719897183891384 in 81.0353s
Beginning eva

Negative ELBO at step 2500: 31.393298894166385 in 234.0947s
Beginning evaluation 14
Evaluation 14: Next query is [[0.89667129 0.99033895 0.21689698]
 [0.7706745  0.72052752 0.63341482]]
Evaluation 14: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 23743.303871385357 in 0.5944s
Negative ELBO at step 500: 436.49998369216564 in 256.9824s
Negative ELBO at step 1000: 191.83191515987096 in 255.7546s
Negative ELBO at step 1500: 107.85991989387145 in 251.8258s
Negative ELBO at step 2000: 58.328871764510396 in 246.4187s
Negative ELBO at step 2500: 41.368681481522955 in 247.7443s
Beginning evaluation 15
Evaluation 15: Next query is [[0.79920259 0.63044794 0.87428797]
 [0.24768502 0.31823351 0.85877747]]
Evaluation 15: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 32387.840946418954 in 0.6192s
Negative ELBO at step 500: 507.0541459036616 in 280.6521s
Negative ELBO at step 1000: 199.06559773317832 in 279.2624s
Negative ELBO at step 1500:

Negative ELBO at step 500: 191.44619292614797 in 186.4441s
Negative ELBO at step 1000: 82.2429870000664 in 189.0284s
Negative ELBO at step 1500: 42.86405620828353 in 190.7339s
Negative ELBO at step 2000: 28.623549101576472 in 191.1196s
Negative ELBO at step 2500: 25.366771340139845 in 187.8005s
Beginning evaluation 9
Evaluation 9: Next query is [[0.56804456 0.92559664 0.07103606]
 [0.27798994 0.79328167 0.65997055]]
Evaluation 9: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 25805.285067691908 in 0.4268s
Negative ELBO at step 500: 283.11078482505 in 199.5357s
Negative ELBO at step 1000: 111.29369961253582 in 194.3232s
Negative ELBO at step 1500: 64.17705963161323 in 199.0939s
Negative ELBO at step 2000: 40.40461973235741 in 197.6195s
Negative ELBO at step 2500: 29.60403876987674 in 196.5172s
Beginning evaluation 10
Evaluation 10: Next query is [[0.3798939  0.56045059 0.66821823]
 [0.96391299 0.30342126 0.26668018]]
Evaluation 10: Training model
Indifferen

Beginning evaluation 3
Evaluation 3: Next query is [[0.15949655 0.19131256 0.54934832]
 [0.57499033 0.73993748 0.70466465]]
Evaluation 3: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 729.1742622527129 in 0.7739s
Negative ELBO at step 500: 20.999628748067213 in 122.2817s
Negative ELBO at step 1000: 8.506943564453001 in 119.1726s
Negative ELBO at step 1500: 6.495390972528267 in 116.0783s
Negative ELBO at step 2000: 5.5751000745533945 in 126.1923s
Negative ELBO at step 2500: 5.127143082345237 in 118.6762s
Beginning evaluation 4
Evaluation 4: Next query is [[0.77815675 0.87001215 0.97861834]
 [0.14297878 0.35455447 0.66780345]]
Evaluation 4: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 251.79355952336925 in 0.3168s
Negative ELBO at step 500: 13.9884909675068 in 129.0723s
Negative ELBO at step 1000: 7.673384449011448 in 133.8298s
Negative ELBO at step 1500: 6.394011964903868 in 138.5595s
Negative ELBO at step 2000: 6.1760665586

Negative ELBO at step 0: 124791.21240970556 in 0.7704s
Negative ELBO at step 500: 2192.8061096775427 in 262.8392s
Negative ELBO at step 1000: 761.4761145404964 in 266.6468s
Negative ELBO at step 1500: 451.1108664198015 in 266.8553s
Negative ELBO at step 2000: 257.26119372490155 in 262.6250s
Negative ELBO at step 2500: 169.28073188323015 in 264.5327s
Beginning evaluation 19
Evaluation 19: Next query is [[0.063369   0.40985745 0.72250009]
 [0.84055719 0.08757178 0.97551218]]
Evaluation 19: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 137786.8883608765 in 0.7290s
Negative ELBO at step 500: 2125.6009469709134 in 272.1197s
Negative ELBO at step 1000: 930.1423476984058 in 271.7647s
Negative ELBO at step 1500: 452.4792072343408 in 273.7551s
Negative ELBO at step 2000: 296.0542166585242 in 273.4893s
Negative ELBO at step 2500: 177.09967030589067 in 272.6000s
Beginning run 3
Indifference_threshold is trainable.
Negative ELBO at step 0: 7.0263334067224585 in 0.143

Negative ELBO at step 2500: 35.85661674611275 in 199.9837s
Beginning evaluation 13
Evaluation 13: Next query is [[0.93686283 0.77386297 0.40563389]
 [0.45849116 0.08161389 0.66102798]]
Evaluation 13: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 82278.99473080291 in 0.7554s
Negative ELBO at step 500: 1183.5324040861071 in 209.7982s
Negative ELBO at step 1000: 391.3808206288098 in 210.2964s
Negative ELBO at step 1500: 175.06082403795043 in 211.2483s
Negative ELBO at step 2000: 137.44505482585555 in 213.5085s
Negative ELBO at step 2500: 68.88664529301332 in 207.9849s
Beginning evaluation 14
Evaluation 14: Next query is [[0.26455561 0.77423369 0.45615033]
 [0.12925762 0.50268559 0.18631862]]
Evaluation 14: Training model
Indifference_threshold is trainable.
Negative ELBO at step 0: 81923.92647814724 in 0.5975s
Negative ELBO at step 500: 1294.8476120127943 in 222.0861s
Negative ELBO at step 1000: 539.0251071345306 in 216.9956s
Negative ELBO at step 1500: 237.

KeyboardInterrupt: 

In [ ]:
print_summary(model)

In [ ]:
pickle.dump((X_results, y_results, best_guess_results), open(results_dir + "Xybestguess.p", "wb"))

In [ ]:
def dist(x, y):
    """
    x and y have shape (..., input_dims)
    """
    return np.sqrt(np.sum((x - y) * (x - y), axis=-1))

xx = uniform_grid(input_dims, num_discrete_per_dim, low=objective_low, high=objective_high)
global_min = xx[np.argmin(objective(xx))][0]

for i in range(best_guess_results.shape[0]):
    diff_from_min = dist(best_guess_results[i], global_min)
    
    x_axis = list(range(num_combs+1, num_combs+1+num_evals))
    plt.figure(figsize=(12, 6))
    plt.plot(x_axis, diff_from_min, 'kx', mew=2)
    plt.xticks(x_axis)
    plt.xlabel('Evaluations', fontsize=18)
    plt.ylabel('Best guess distance', fontsize=16)
    plt.title("Run %s" % i)
    plt.show()